In [1]:
import nltk

nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# import thư viện
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

# đọc file json các mẫu câu hỏi và câu trả lời
words = []
classes = []
documents = []
ignore_words = ["?", "!"]

data_file = open("/content/drive/MyDrive/code/Chatbot/intents.json").read()
intents = json.loads(data_file)

# # Tiền xử lý dữ liệu
# for intent in intents["intents"]:
#     for pattern in intent["patterns"]:

#         # tokenize each word
#         w = nltk.word_tokenize(pattern)
#         words.extend(w)
#         # add documents in the corpus
#         documents.append((w, intent["tag"]))

#         # add to our classes list
#         if intent["tag"] not in classes:
#             classes.append(intent["tag"])
# Tiền xử lý dữ liệu
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        # pattern is a list, so we need to join it into a string
        pattern_str = " ".join(
            pattern
        )  # Join the list of strings into a single string with spaces in between
        # tokenize each word
        w = nltk.word_tokenize(pattern_str)
        words.extend(w)
        # add documents in the corpus
        documents.append((w, intent["tag"]))

        # add to our classes list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

# Các từ được lemmatized và chuyển đổi thành chữ thường, sau đó loại bỏ các từ không quan trọng như dấu chấm hỏi và dấu chấm than.
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print(len(documents), "documents")
# classes = intents
print(len(classes), "classes", classes)
# words = all words, vocabulary
print(len(words), "unique lemmatized words", words)


pickle.dump(words, open("/content/drive/MyDrive/code/Chatbot/words.pkl", "wb"))
pickle.dump(classes, open("/content/drive/MyDrive/code/Chatbot/classes.pkl", "wb"))


# Tạo dữ liệu huấn luyện
# tạo training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# tập huấn luyện, túi từ cho mỗi câu
for doc in documents:
    # Khởi tạo
    bag = []
    # danh sách các từ được mã hóa cho mẫu
    pattern_words = doc[0]
    # bổ nghĩa hóa từng từ - tạo từ cơ sở, nhằm thể hiện các từ liên quan
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # tạo mảng từ của chúng ta bằng 1, nếu tìm thấy từ khớp trong mẫu hiện tại
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # đầu ra là '0' cho mỗi thẻ và '1' cho thẻ hiện tại (cho mỗi mẫu)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# xáo trộn dữ liện huấn luyện và biến đổi chúng thành np.array
random.shuffle(training)

# Tách các tính năng (X) và nhãn (Y) khỏi dữ liệu huấn luyện
train_x = np.array([x[0] for x in training])  # X - patterns
train_y = np.array([x[1] for x in training])  # Y - intents

print("Training data created")


# Tạo model - 3 layers.  128 neurons, 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))

# Compile model. Giảm độ dốc ngẫu nhiên với độ dốc tăng tốc Nesterov cho kết quả tốt cho mô hình này
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

# lưu mmodel
hist = model.fit(
    np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1
)
model.save("/content/drive/MyDrive/code/Chatbot/chatbot_model.h5", hist)

print("model created and saved")

67 documents
50 classes ['bạn_có_adidas_superstar_không', 'bạn_có_nike_air_max_90_essential_không', 'bạn_có_puma_suede_classic+_không', 'bạn_có_reebok_classic_không', 'có_adidas_không', 'có_nike_không', 'có_puma_không', 'có_reebok_không', 'có_tất_cả_bao_nhiêu_khuyến_mãi', 'có_tất_cả_bao_nhiêu_kích_thước_giày', 'có_tất_cả_bao_nhiêu_màu_giày', 'có_tất_cả_bao_nhiêu_mặc_hàng', 'có_tất_cả_bao_nhiêu_thương_hiệu_giày', 'giày_adidas_superstar_giá_bao_nhiêu', 'giày_adidas_superstar_màu_black_có_đẹp_không', 'giày_adidas_superstar_màu_blue_có_đẹp_không', 'giày_adidas_superstar_màu_grey_có_đẹp_không', 'giày_adidas_superstar_màu_light_blue_có_đẹp_không', 'giày_adidas_superstar_màu_orange_có_đẹp_không', 'giày_adidas_superstar_màu_red_có_đẹp_không', 'giày_adidas_superstar_màu_yellow_có_đẹp_không', 'giày_nike_air_max_90_essential_giá_bao_nhiêu', 'giày_nike_air_max_90_essential_màu_black_có_đẹp_không', 'giày_nike_air_max_90_essential_màu_blue_có_đẹp_không', 'giày_nike_air_max_90_essential_màu_

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Test

In [ ]:
from tensorflow.keras.models import load_model

# Load mô hình và các dữ liệu cần thiết từ các tệp đã lưu trữ trước đó
model = load_model("/content/drive/MyDrive/code/Chatbot/chatbot_model.h5")
intents = json.loads(open("/content/drive/MyDrive/code/Chatbot/intents.json").read())
words = pickle.load(open("/content/drive/MyDrive/code/Chatbot/words.pkl", "rb"))
classes = pickle.load(open("/content/drive/MyDrive/code/Chatbot/classes.pkl", "rb"))

# Định nghĩa hàm để làm sạch câu nhập từ người dùng
lemmatizer = WordNetLemmatizer()


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# Định nghĩa hàm bow (bag of words) để chuyển đổi câu nhập thành một vector các từ
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)


# Định nghĩa hàm predict_class để dự đoán nhãn (intent) của câu nhập:
def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]["intent"]
    print(tag)
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result


def chatbot_response(msg):
    ints = predict_class(msg, model)
    if ints:
        res = getResponse(ints, intents)
        return res
    else:
        return "Xin lỗi, tôi không hiểu câu của bạn. Vui lòng thử lại."


# Tạo vòng lặp vô hạn để cho phép người dùng tương tác với chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break
    response = chatbot_response(user_input)
    print("Bot:", response)

You: chào
1/1 [==============================] - 0s 86ms/step
greeting
Bot: Xin chào, Shop có thể giúp gì cho bạn
You: Có tất cả bao nhiêu mặc hàng
1/1 [==============================] - 0s 19ms/step
có_tất_cả_bao_nhiêu_mặc_hàng
Bot: Tổng số giày hiện có là 4. Danh sách các giày: Nike Air Max 90 Essential, Puma Suede Classic+, Adidas Superstar, Reebok Classic.
You: Có tất cả bao nhiêu thương hiệu giày
1/1 [==============================] - 0s 20ms/step
có_tất_cả_bao_nhiêu_thương_hiệu_giày
Bot: Tổng số thương hiệu giày hiện có là 4. Danh sách các thương hiệu giày: Reebok, Puma, Nike, Adidas.
You: Có Nike không
1/1 [==============================] - 0s 31ms/step
có_nike_không
Bot: Có, chúng tôi có Nike Air Max 90 Essential từ thương hiệu Nike.
You: Nike Air Max 90 Essential màu Grey
1/1 [==============================] - 0s 39ms/step
giày_nike_air_max_90_essential_màu_grey_có_đẹp_không
Bot: Có, giày Nike Air Max 90 Essential màu Grey là một trong những màu được ưa chuộng.
You